In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [3]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final = pd.read_parquet(DATOS_DIR+'/FE_02_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [4]:
#weight = df_final.groupby(['product_id', df_final.index])['weight'].transform('mean')

df_final.groupby('product_id')['weight'].transform(scale_group)


periodo
2017-01     0.222190
2017-01     1.444379
2017-01    -0.111242
2017-01    -0.222336
2017-01    22.111242
             ...    
2019-12     0.384748
2019-12    -0.153899
2019-12     0.615252
2019-12     0.153899
2019-12     0.153899
Freq: M, Name: weight, Length: 2293481, dtype: float64

In [5]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


#Filtro de no compradores
#Step 1: Ensure the index is a datetime type
df_final.index = df_final.index.to_timestamp()
# Step 2: Determine the last date and calculate the date 3 months prior
ls_date  = df_final.index.max()
three_months_prior = ls_date - pd.DateOffset(months=3)

# Step 3: Filter the dataframe to include onl+y rows within the last 3 months
last_3_months_df = df_final[df_final.index >= three_months_prior]

# Step 4: Identify the unique client_id that have purchased within this period
active_clients = last_3_months_df['customer_id'].unique()

# Step 5: Filter the original dataframe to include only these client_id
df_final = df_final[df_final['customer_id'].isin(active_clients)]

df_final.index = pd.PeriodIndex(df_final.index, freq='M')


#Filtro test
df_final = df_final[df_final['product_id'] < 20013]

weight= df_final[['weight','product_id']]
df_final.drop(columns=['weight'], inplace=True)


In [6]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 10000079,
        #'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    weight_p= weight[weight['product_id'] == producto]
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['tn_2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse',
                                                      weights=weight_p['weight'])
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20007
Epoch 1/10


d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 991us/step - loss: -0.9087
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - loss: -7.6636
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: -15.7159
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: -26.1966
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - loss: -33.0268
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - loss: -37.7819
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step - loss: -45.1817
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - loss: -48.1698
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - loss: -65.4192
Epoch 10/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - loss: -65.6722
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	validation's rmse: 5.01995	validation's multinacional_metric: 0.000192995
Training until validation scores don't improve for 10 rounds
Early stopping, best itera

C:\Users\Admin\AppData\Local\Temp\ipykernel_11460\376699480.py:70: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_11460\376699480.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Seq

111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - loss: -0.4298
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: -5.5996
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - loss: -11.4734
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: -16.4966
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step - loss: -22.2192
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -25.1333
Epoch 7/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - loss: -31.8284
Epoch 8/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: -37.7445
Epoch 9/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - loss: -43.4526
Epoch 10/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: -50.1856
111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	validation's rmse: 3.27304	validation's multinacional_metric: 0.0198166
Training until validation scores don't improve for 10 rounds
Early stopping, best iteratio

...

119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 706us/step - loss: -1.7705
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: -19.4984
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - loss: -44.8770
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - loss: -65.5351
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step - loss: -82.7247
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: -83.6912
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: -95.3805
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: -103.3199
Epoch 9/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step - loss: -137.2907
Epoch 10/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -148.3090
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	validation's rmse: 11.034	validation's multinacional_metric: 0.193049
Training until validation scores don't improve for 10 rounds
Early stopping, best iterati

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 777us/step - loss: -0.1482
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -5.2723
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - loss: -13.0410
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: -20.1260
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - loss: -22.8093
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - loss: -35.7309
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - loss: -35.8002
Epoch 8/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - loss: -38.5243
Epoch 9/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -44.4961
Epoch 10/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - loss: -52.8649
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	validation's rmse: 4.39012	validation's multinacional_metric: 0.00239828
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - loss: -1.9847
Epoch 2/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - loss: -13.3190
Epoch 3/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - loss: -25.7025
Epoch 4/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: -34.7231
Epoch 5/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step - loss: -50.2164
Epoch 6/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - loss: -55.2429
Epoch 7/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: -59.0541 
Epoch 8/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - loss: -70.7670
Epoch 9/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: -85.9245
Epoch 10/10
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - loss: -93.9783
110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[16]	validation's rmse: 3.23561	validation's multinacional_metric: 0.00694329
Training until validation scores don't improve for 10 rounds
Early stopping, best itera

In [7]:
len(weight_p['weight'])
len(df_producto)

3506

In [8]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0019-prediccion-rmse_scaled_CORRECT2-product_id_LSTM-No_Buyer_weights.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  0.006943293130107512
